<a href="https://colab.research.google.com/github/YOUR_USERNAME/Digital-Finance-Introduction/blob/main/day_03/notebooks/NB07_Blockchain_Transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NB07: Blockchain Transactions

**Topic:** 3.3 - Wallets, Transactions, and UX

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand Transaction Structure**: Identify and explain the key components of a blockchain transaction
2. **Read Blockchain Data**: Connect to Ethereum nodes and retrieve real-time blockchain information
3. **Analyze Real Transactions**: Parse and interpret actual transaction data from the Ethereum network
4. **Query Account Information**: Retrieve and analyze account balances and transaction history
5. **Appreciate UX Challenges**: Understand the complexity gap between raw blockchain data and user-friendly interfaces

## Section 1: Setup

We'll use the `web3.py` library to interact with the Ethereum blockchain. This notebook includes fallback data to ensure all examples work even without an active internet connection.

In [ ]:
# Install required packages
!pip install web3 -q

In [ ]:
# Import required libraries
from web3 import Web3
from datetime import datetime
import json
from typing import Dict, Any, Optional

# Flag to track if we're using live data or fallback
USING_LIVE_DATA = True

print("Libraries imported successfully!")

# Get Web3.py version (works with 5.x+ and older versions)
try:
    from importlib.metadata import version
    web3_version = version('web3')
except:
    try:
        web3_version = Web3.__version__
    except AttributeError:
        web3_version = "unknown"
print(f"Web3.py version: {web3_version}")

### Fallback Data

We maintain static copies of blockchain data for offline use. This ensures the notebook works even when RPC endpoints are unavailable.

In [ ]:
# Fallback data structures
FALLBACK_BLOCK = {
    'number': 18500000,
    'hash': '0x1234567890abcdef1234567890abcdef1234567890abcdef1234567890abcdef',
    'parentHash': '0xabcdef1234567890abcdef1234567890abcdef1234567890abcdef1234567890',
    'timestamp': 1698768000,
    'transactions': [
        '0x5c504ed432cb51138bcf09aa5e8a410dd4a1e204ef84bfed1be16dfba1b22060',
        '0x6c504ed432cb51138bcf09aa5e8a410dd4a1e204ef84bfed1be16dfba1b22061',
    ],
    'gasUsed': 15000000,
    'gasLimit': 30000000,
    'baseFeePerGas': 20000000000,
    'difficulty': 0,
    'miner': '0x0000000000000000000000000000000000000000',
    'size': 125000,
    'transactionCount': 150
}

FALLBACK_TRANSACTION = {
    'hash': '0x5c504ed432cb51138bcf09aa5e8a410dd4a1e204ef84bfed1be16dfba1b22060',
    'blockNumber': 18500000,
    'from': '0x742d35Cc6634C0532925a3b844Bc9e7595f0bEb',
    'to': '0x5AEDA56215b167893e80B4fE645BA6d5Bab767DE',
    'value': 1500000000000000000,  # 1.5 ETH in Wei
    'gas': 21000,
    'gasPrice': 25000000000,  # 25 Gwei
    'maxFeePerGas': 30000000000,
    'maxPriorityFeePerGas': 2000000000,
    'nonce': 42,
    'input': '0x',
    'transactionIndex': 15,
    'type': '0x2'  # EIP-1559 transaction
}

FALLBACK_RECEIPT = {
    'transactionHash': '0x5c504ed432cb51138bcf09aa5e8a410dd4a1e204ef84bfed1be16dfba1b22060',
    'status': 1,  # Success
    'gasUsed': 21000,
    'effectiveGasPrice': 23000000000,
    'cumulativeGasUsed': 450000
}

# Famous Ethereum addresses with known balances (approximate)
FAMOUS_ADDRESSES = {
    'vitalik': {
        'address': '0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045',
        'name': "Vitalik Buterin's Public Address",
        'fallback_balance': 2500000000000000000  # ~2.5 ETH
    },
    'ethereum_foundation': {
        'address': '0xde0B295669a9FD93d5F28D9Ec85E40f4cb697BAe',
        'name': 'Ethereum Foundation',
        'fallback_balance': 285000000000000000000  # ~285 ETH
    },
    'null_address': {
        'address': '0x0000000000000000000000000000000000000000',
        'name': 'Null Address (Burn Address)',
        'fallback_balance': 0
    }
}

print("Fallback data structures initialized")

## Section 2: Connecting to Ethereum

To interact with the Ethereum blockchain, we need to connect to a node. There are several options:

1. **Public RPC Endpoints**: Free services like LlamaRPC or Cloudflare
2. **Managed Services**: Infura, Alchemy, QuickNode (require API keys)
3. **Local Node**: Run your own Ethereum node (geth, erigon)

We'll use public RPC endpoints with fallback handling.

In [ ]:
# List of public RPC endpoints to try
RPC_ENDPOINTS = [
    'https://eth.llamarpc.com',
    'https://cloudflare-eth.com',
    'https://rpc.ankr.com/eth',
    'https://eth-mainnet.public.blastapi.io'
]

def connect_to_ethereum():
    """
    Attempt to connect to Ethereum using multiple RPC endpoints.
    Returns Web3 instance or None if all connections fail.
    """
    global USING_LIVE_DATA
    
    for endpoint in RPC_ENDPOINTS:
        try:
            w3 = Web3(Web3.HTTPProvider(endpoint, request_kwargs={'timeout': 10}))
            
            # Test connection by getting latest block
            block_number = w3.eth.block_number
            
            print(f"✓ Successfully connected to: {endpoint}")
            print(f"✓ Latest block number: {block_number:,}")
            print(f"✓ Network ID: {w3.eth.chain_id}")
            USING_LIVE_DATA = True
            return w3
            
        except Exception as e:
            print(f"✗ Failed to connect to {endpoint}: {str(e)[:50]}")
            continue
    
    print("\n⚠ All RPC endpoints failed. Using fallback data.")
    USING_LIVE_DATA = False
    return None

# Attempt connection
w3 = connect_to_ethereum()

if USING_LIVE_DATA:
    print(f"\n📡 Connected to Ethereum Mainnet")
    print(f"   Chain ID: {w3.eth.chain_id}")
    print(f"   Client version: {w3.client_version[:50]}...")
else:
    print(f"\n💾 Using offline fallback data")

## Section 3: Transaction Anatomy

An Ethereum transaction contains several key fields:

- **from**: Address of the sender
- **to**: Address of the recipient (null for contract creation)
- **value**: Amount of ETH to transfer (in Wei, where 1 ETH = 10^18 Wei)
- **gas**: Maximum gas units the transaction can consume
- **gasPrice** or **maxFeePerGas**: Price per gas unit (in Wei)
- **nonce**: Transaction counter for the sender's address
- **data**: Input data for contract interaction (or empty for simple transfers)
- **type**: Transaction type (0x0 = Legacy, 0x2 = EIP-1559)

Let's examine a real transaction:

In [ ]:
def get_transaction(tx_hash: str) -> Optional[Dict[str, Any]]:
    """
    Fetch transaction details with fallback support.
    """
    if USING_LIVE_DATA and w3:
        try:
            tx = w3.eth.get_transaction(tx_hash)
            # Convert AttributeDict to regular dict for easier display
            return dict(tx)
        except Exception as e:
            print(f"⚠ Failed to fetch transaction: {str(e)[:50]}")
            print("  Using fallback data...")
    
    return FALLBACK_TRANSACTION.copy()

def format_transaction(tx: Dict[str, Any]) -> None:
    """
    Display transaction details in a readable format.
    """
    print("=" * 80)
    print("TRANSACTION DETAILS")
    print("=" * 80)
    
    # Basic identification
    print(f"\n📝 Transaction Hash:")
    print(f"   {tx.get('hash', 'N/A')}")
    print(f"\n📦 Block Number: {tx.get('blockNumber', 'N/A'):,}")
    print(f"📍 Position in Block: {tx.get('transactionIndex', 'N/A')}")
    
    # Addresses
    print(f"\n👤 From Address:")
    print(f"   {tx.get('from', 'N/A')}")
    print(f"\n👥 To Address:")
    print(f"   {tx.get('to', 'N/A')}")
    
    # Value
    value_wei = tx.get('value', 0)
    value_eth = value_wei / 10**18
    print(f"\n💰 Value:")
    print(f"   {value_wei:,} Wei")
    print(f"   {value_eth:.6f} ETH")
    
    # Gas details
    print(f"\n⛽ Gas Information:")
    print(f"   Gas Limit: {tx.get('gas', 'N/A'):,} units")
    
    if 'gasPrice' in tx:
        gas_price_wei = tx['gasPrice']
        gas_price_gwei = gas_price_wei / 10**9
        print(f"   Gas Price: {gas_price_wei:,} Wei ({gas_price_gwei:.2f} Gwei)")
    
    if 'maxFeePerGas' in tx:
        max_fee = tx['maxFeePerGas'] / 10**9
        print(f"   Max Fee Per Gas: {max_fee:.2f} Gwei")
    
    if 'maxPriorityFeePerGas' in tx:
        priority_fee = tx['maxPriorityFeePerGas'] / 10**9
        print(f"   Max Priority Fee: {priority_fee:.2f} Gwei")
    
    # Calculate maximum transaction cost
    gas_limit = tx.get('gas', 0)
    gas_price = tx.get('gasPrice', tx.get('maxFeePerGas', 0))
    max_cost_wei = gas_limit * gas_price
    max_cost_eth = max_cost_wei / 10**18
    print(f"   Max Transaction Cost: {max_cost_eth:.6f} ETH")
    
    # Transaction metadata
    print(f"\n🔢 Nonce: {tx.get('nonce', 'N/A')}")
    print(f"📋 Transaction Type: {tx.get('type', 'N/A')}")
    
    # Data field
    data = tx.get('input', '0x')
    if data == '0x' or data == '':
        print(f"\n💬 Data: (empty - simple ETH transfer)")
    else:
        print(f"\n💬 Data: {data[:66]}..." if len(data) > 66 else f"\n💬 Data: {data}")
        print(f"   (Contains {(len(data)-2)//2} bytes of contract interaction data)")
    
    print("\n" + "=" * 80)

# Example: Fetch a well-known transaction
# This is a famous early Ethereum transaction
EXAMPLE_TX_HASH = '0x5c504ed432cb51138bcf09aa5e8a410dd4a1e204ef84bfed1be16dfba1b22060'

transaction = get_transaction(EXAMPLE_TX_HASH)
format_transaction(transaction)

### Transaction Lifecycle

A transaction goes through several stages:

1. **Creation**: User creates and signs transaction with their private key
2. **Broadcast**: Transaction is sent to the network's mempool
3. **Pending**: Transaction waits in mempool for inclusion
4. **Inclusion**: Validator includes transaction in a block
5. **Confirmation**: Block is added to the canonical chain
6. **Finality**: After ~15 minutes (2 epochs), block is finalized

Let's get the transaction receipt to see the execution result:

In [ ]:
def get_transaction_receipt(tx_hash: str) -> Optional[Dict[str, Any]]:
    """
    Fetch transaction receipt with fallback support.
    """
    if USING_LIVE_DATA and w3:
        try:
            receipt = w3.eth.get_transaction_receipt(tx_hash)
            return dict(receipt)
        except Exception as e:
            print(f"⚠ Failed to fetch receipt: {str(e)[:50]}")
            print("  Using fallback data...")
    
    return FALLBACK_RECEIPT.copy()

def format_receipt(receipt: Dict[str, Any]) -> None:
    """
    Display transaction receipt in a readable format.
    """
    print("=" * 80)
    print("TRANSACTION RECEIPT")
    print("=" * 80)
    
    status = receipt.get('status', 0)
    print(f"\n✅ Status: {'SUCCESS' if status == 1 else 'FAILED'}")
    
    gas_used = receipt.get('gasUsed', 0)
    print(f"\n⛽ Gas Used: {gas_used:,} units")
    
    if 'effectiveGasPrice' in receipt:
        effective_price = receipt['effectiveGasPrice']
        effective_gwei = effective_price / 10**9
        print(f"💸 Effective Gas Price: {effective_gwei:.2f} Gwei")
        
        # Calculate actual transaction fee
        fee_wei = gas_used * effective_price
        fee_eth = fee_wei / 10**18
        print(f"💰 Transaction Fee: {fee_eth:.6f} ETH (${fee_eth * 2000:.2f} at $2000/ETH)")
    
    print(f"\n📊 Cumulative Gas Used in Block: {receipt.get('cumulativeGasUsed', 'N/A'):,}")
    
    print("\n" + "=" * 80)

# Get receipt for the same transaction
receipt = get_transaction_receipt(EXAMPLE_TX_HASH)
format_receipt(receipt)

## Section 4: Reading Blockchain Data

Blocks are the fundamental data structure of the blockchain. Each block contains:

- **Block Header**: Metadata about the block
- **Transactions**: List of all transactions in the block
- **State Root**: Merkle root of the entire state

Let's explore block data:

In [ ]:
def get_latest_block() -> Optional[Dict[str, Any]]:
    """
    Fetch the latest block with fallback support.
    """
    if USING_LIVE_DATA and w3:
        try:
            block = w3.eth.get_block('latest')
            return dict(block)
        except Exception as e:
            print(f"⚠ Failed to fetch latest block: {str(e)[:50]}")
            print("  Using fallback data...")
    
    return FALLBACK_BLOCK.copy()

def format_block(block: Dict[str, Any]) -> None:
    """
    Display block information in a readable format.
    """
    print("=" * 80)
    print("BLOCK INFORMATION")
    print("=" * 80)
    
    print(f"\n🧱 Block Number: {block.get('number', 'N/A'):,}")
    print(f"\n#️⃣ Block Hash:")
    print(f"   {block.get('hash', 'N/A')}")
    print(f"\n⬅️ Parent Hash:")
    print(f"   {block.get('parentHash', 'N/A')}")
    
    # Timestamp
    timestamp = block.get('timestamp', 0)
    if timestamp:
        dt = datetime.fromtimestamp(timestamp)
        print(f"\n⏰ Timestamp: {dt.strftime('%Y-%m-%d %H:%M:%S UTC')}")
        print(f"   Unix: {timestamp}")
    
    # Miner/Validator
    print(f"\n⛏️ Miner/Validator:")
    print(f"   {block.get('miner', 'N/A')}")
    
    # Transactions
    tx_count = len(block.get('transactions', []))
    print(f"\n📦 Transactions: {tx_count}")
    
    # Gas information
    gas_used = block.get('gasUsed', 0)
    gas_limit = block.get('gasLimit', 0)
    gas_utilization = (gas_used / gas_limit * 100) if gas_limit > 0 else 0
    
    print(f"\n⛽ Gas Information:")
    print(f"   Gas Used: {gas_used:,}")
    print(f"   Gas Limit: {gas_limit:,}")
    print(f"   Utilization: {gas_utilization:.2f}%")
    
    if 'baseFeePerGas' in block:
        base_fee_gwei = block['baseFeePerGas'] / 10**9
        print(f"   Base Fee: {base_fee_gwei:.2f} Gwei")
    
    # Block size
    if 'size' in block:
        size_kb = block['size'] / 1024
        print(f"\n📏 Block Size: {size_kb:.2f} KB")
    
    # Difficulty (0 after The Merge)
    print(f"\n🎯 Difficulty: {block.get('difficulty', 'N/A')}")
    
    print("\n" + "=" * 80)

# Fetch and display latest block
latest_block = get_latest_block()
format_block(latest_block)

### Transactions in a Block

Let's examine the transactions within a block:

In [ ]:
def analyze_block_transactions(block: Dict[str, Any]) -> None:
    """
    Analyze transactions in a block.
    """
    transactions = block.get('transactions', [])
    
    if not transactions:
        print("No transactions in this block.")
        return
    
    print(f"\n📊 TRANSACTION ANALYSIS")
    print(f"Total transactions: {len(transactions)}")
    
    # If transactions are just hashes, show them
    if isinstance(transactions[0], str):
        print(f"\nFirst 5 transaction hashes:")
        for i, tx_hash in enumerate(transactions[:5], 1):
            print(f"  {i}. {tx_hash}")
        
        if len(transactions) > 5:
            print(f"  ... and {len(transactions) - 5} more")
    
    # If we have full transaction objects, analyze them
    else:
        total_value = sum(tx.get('value', 0) for tx in transactions)
        total_value_eth = total_value / 10**18
        
        print(f"\n💰 Total ETH transferred: {total_value_eth:.2f} ETH")
        
        # Find largest transaction
        if transactions:
            largest_tx = max(transactions, key=lambda tx: tx.get('value', 0))
            largest_value_eth = largest_tx.get('value', 0) / 10**18
            print(f"\n🏆 Largest transaction: {largest_value_eth:.4f} ETH")
            print(f"   From: {largest_tx.get('from', 'N/A')[:20]}...")
            print(f"   To: {largest_tx.get('to', 'N/A')[:20]}...")

analyze_block_transactions(latest_block)

## Section 5: Account and Balance Queries

The Ethereum blockchain maintains a state database of all accounts and their balances. Let's query some famous addresses:

In [ ]:
def get_balance(address: str, fallback_balance: int = 0) -> int:
    """
    Get ETH balance for an address with fallback support.
    Returns balance in Wei.
    """
    if USING_LIVE_DATA and w3:
        try:
            balance = w3.eth.get_balance(address)
            return balance
        except Exception as e:
            print(f"⚠ Failed to fetch balance: {str(e)[:50]}")
            print(f"  Using fallback data...")
    
    return fallback_balance

def format_balance(address: str, name: str, balance_wei: int) -> None:
    """
    Display balance information in a readable format.
    """
    balance_eth = balance_wei / 10**18
    balance_usd = balance_eth * 2000  # Approximate at $2000/ETH
    
    print(f"\n{'='*80}")
    print(f"💼 {name}")
    print(f"{'='*80}")
    print(f"\n📍 Address: {address}")
    print(f"\n💰 Balance:")
    print(f"   {balance_wei:,} Wei")
    print(f"   {balance_eth:,.6f} ETH")
    print(f"   ≈ ${balance_usd:,.2f} USD (at $2000/ETH)")
    print(f"\n{'='*80}")

# Query famous addresses
print("\n🔍 QUERYING FAMOUS ETHEREUM ADDRESSES\n")

for key, info in FAMOUS_ADDRESSES.items():
    balance = get_balance(info['address'], info['fallback_balance'])
    format_balance(info['address'], info['name'], balance)

### Understanding Wei and ETH

Ethereum uses Wei as its base unit to avoid floating-point precision issues:

- 1 ETH = 1,000,000,000,000,000,000 Wei (10^18)
- 1 Gwei = 1,000,000,000 Wei (10^9) - commonly used for gas prices
- 1 Szabo = 1,000,000 Wei (10^6)
- 1 Finney = 1,000,000,000,000,000 Wei (10^15)

In [ ]:
def demonstrate_wei_conversions():
    """
    Demonstrate Wei conversions.
    """
    print("\n" + "=" * 80)
    print("WEI CONVERSION EXAMPLES")
    print("=" * 80)
    
    examples = [
        (1, "1 Wei"),
        (10**9, "1 Gwei (typical gas price unit)"),
        (10**18, "1 ETH"),
        (21000 * 25 * 10**9, "Transaction cost: 21,000 gas at 25 Gwei"),
        (1500000000000000000, "1.5 ETH (typical transaction amount)"),
    ]
    
    for wei_amount, description in examples:
        eth_amount = wei_amount / 10**18
        gwei_amount = wei_amount / 10**9
        
        print(f"\n{description}:")
        print(f"  Wei:  {wei_amount:,}")
        
        if wei_amount >= 10**9:
            print(f"  Gwei: {gwei_amount:,.6f}")
        
        if wei_amount >= 10**18:
            print(f"  ETH:  {eth_amount:,.6f}")
    
    print("\n" + "=" * 80)

demonstrate_wei_conversions()

## Section 6: The User Experience Gap

As we've seen, raw blockchain data is complex and technical. Let's compare the raw transaction data with what users see in wallets:

### Raw Blockchain Data:
- Addresses: `0x742d35Cc6634C0532925a3b844Bc9e7595f0bEb`
- Values: `1500000000000000000 Wei`
- Gas: `21000 units at 25000000000 Wei per unit`
- Nonce: `42`
- Block confirmations: Monitor yourself

### What Wallets Show:
- Contact names: "Alice" instead of `0x742d35...`
- Simple amounts: "1.5 ETH" instead of Wei
- Estimated fees: "$2.50" instead of gas calculations
- Confirmation status: "✓ Confirmed" with progress bar
- Transaction history with nice formatting

### Key UX Challenges:

1. **Irreversible Mistakes**: One wrong character in an address = lost funds forever
2. **Gas Price Complexity**: Users must understand gas markets to avoid overpaying
3. **No Customer Support**: No bank to call if something goes wrong
4. **Long Addresses**: 42-character hexadecimal strings are error-prone
5. **Confirmation Times**: Variable and unpredictable transaction finality
6. **MEV and Front-running**: Advanced users can exploit transaction ordering
7. **Smart Contract Risk**: Approving contracts can drain your wallet

In [ ]:
def compare_ux_interfaces():
    """
    Compare raw vs wallet-friendly transaction display.
    """
    tx = FALLBACK_TRANSACTION.copy()
    
    print("\n" + "=" * 80)
    print("RAW BLOCKCHAIN DATA (What developers see)")
    print("=" * 80)
    print(f"""
from: 0x742d35Cc6634C0532925a3b844Bc9e7595f0bEb
to: 0x5AEDA56215b167893e80B4fE645BA6d5Bab767DE
value: 1500000000000000000
gas: 21000
gasPrice: 25000000000
nonce: 42
type: 0x2
maxFeePerGas: 30000000000
maxPriorityFeePerGas: 2000000000
    """)
    
    print("\n" + "=" * 80)
    print("WALLET INTERFACE (What users see)")
    print("=" * 80)
    print(f"""
┌──────────────────────────────────────────────┐
│  💸 Send Ethereum                            │
├──────────────────────────────────────────────┤
│                                              │
│  To: Alice (Contact)                         │
│  📋 0x5AED...67DE                            │
│                                              │
│  Amount: 1.5 ETH                             │
│  ≈ $3,000 USD                                │
│                                              │
│  Network Fee: ~$1.31                         │
│  ⚡ Fast (< 30 seconds)                      │
│                                              │
│  Total: 1.500525 ETH ($3,001.31)             │
│                                              │
│  [Cancel]              [Confirm] ✓           │
└──────────────────────────────────────────────┘
    """)
    
    print("\n" + "=" * 80)
    print("KEY UX IMPROVEMENTS WALLETS PROVIDE")
    print("=" * 80)
    print("""
1. Contact Names: Map addresses to human-readable names
2. USD Conversion: Show familiar currency values
3. Fee Estimation: Abstract gas calculations to simple fee tiers
4. Address Validation: Check for common mistakes (checksum validation)
5. Confirmation Tracking: Visual progress of transaction status
6. Transaction History: Organized, searchable past transactions
7. ENS Support: Use names like 'vitalik.eth' instead of addresses
8. Warning Systems: Alert on suspicious transactions or contracts
9. Simulation: Preview transaction outcome before sending
10. Batch Operations: Approve multiple transactions at once
    """)

compare_ux_interfaces()

### Gas Price Market Dynamics

Understanding gas prices is crucial for users but complex:

After EIP-1559 (August 2021), Ethereum uses a two-tier fee structure:

- **Base Fee**: Algorithmically determined, burned (not paid to miners)
- **Priority Fee (Tip)**: Optional extra payment to incentivize inclusion

Total Fee = (Base Fee + Priority Fee) × Gas Used

In [ ]:
def demonstrate_gas_mechanics():
    """
    Demonstrate gas price calculations and scenarios.
    """
    print("\n" + "=" * 80)
    print("GAS PRICE MECHANICS")
    print("=" * 80)
    
    # Typical gas costs for different operations
    operations = [
        ("Simple ETH Transfer", 21000),
        ("ERC-20 Token Transfer", 65000),
        ("Uniswap Swap", 150000),
        ("NFT Mint", 85000),
        ("Deploy Simple Contract", 500000),
    ]
    
    # Different gas price scenarios
    scenarios = [
        ("Low Activity", 10),
        ("Normal", 25),
        ("High Activity", 50),
        ("Network Congestion", 100),
    ]
    
    print("\n📊 Gas Cost by Operation and Network Condition:\n")
    print(f"{'Operation':<25} {'Gas Units':>12}  {'Low':>10}  {'Normal':>10}  {'High':>10}  {'Congested':>12}")
    print("-" * 95)
    
    for op_name, gas_units in operations:
        costs = []
        for scenario_name, gas_price_gwei in scenarios:
            cost_eth = (gas_units * gas_price_gwei * 10**9) / 10**18
            cost_usd = cost_eth * 2000
            costs.append(f"${cost_usd:.2f}")
        
        print(f"{op_name:<25} {gas_units:>12,}  {costs[0]:>10}  {costs[1]:>10}  {costs[2]:>10}  {costs[3]:>12}")
    
    print("\n💡 Key Insights:")
    print("""
1. Gas costs vary dramatically based on network conditions
2. Complex operations (DeFi, NFTs) cost 3-7x more than simple transfers
3. Users must decide: Pay more for speed, or wait for lower prices?
4. Failed transactions still consume gas (and cost money!)
5. Gas price fluctuates constantly - timing matters
    """)
    
    print("\n🎯 Wallet UX Challenge:")
    print("""
How to abstract this complexity while still giving users control?

Common Solutions:
- Simple tiers: "Slow", "Normal", "Fast"
- Time estimates: "Likely in < 30 seconds"
- Gas trackers: "Current gas prices are high, consider waiting"
- Smart defaults: Automatically suggest optimal gas price
- Transaction queuing: Bundle multiple transactions to save gas
    """)

demonstrate_gas_mechanics()

## Section 7: Challenge Exercises

Test your understanding with these challenges:

### Challenge 1: Find High-Value Transaction

Write code to scan the latest block and find any transaction transferring more than 1 ETH.

In [ ]:
def find_high_value_transactions(block: Dict[str, Any], threshold_eth: float = 1.0) -> None:
    """
    Find transactions above a value threshold.
    
    Args:
        block: Block dictionary
        threshold_eth: Minimum ETH value to report
    """
    threshold_wei = int(threshold_eth * 10**18)
    
    print(f"\n🔍 Searching for transactions > {threshold_eth} ETH...\n")
    
    transactions = block.get('transactions', [])
    
    # If we only have hashes, we need to fetch full transaction data
    if transactions and isinstance(transactions[0], str):
        print(f"Block contains {len(transactions)} transactions (hashes only)")
        print("To analyze values, we would need to fetch each transaction individually.")
        print("\nExample approach:")
        print("""for tx_hash in transactions[:10]:  # Check first 10
    tx = w3.eth.get_transaction(tx_hash)
    if tx['value'] > threshold_wei:
        print(f"Found: {tx['value'] / 10**18:.2f} ETH")""")
        return
    
    # If we have full transaction objects
    high_value_txs = [tx for tx in transactions if tx.get('value', 0) > threshold_wei]
    
    if not high_value_txs:
        print(f"No transactions found above {threshold_eth} ETH in this block.")
        return
    
    print(f"Found {len(high_value_txs)} high-value transaction(s):\n")
    
    for i, tx in enumerate(high_value_txs[:5], 1):
        value_eth = tx.get('value', 0) / 10**18
        print(f"{i}. {value_eth:.4f} ETH")
        print(f"   From: {tx.get('from', 'N/A')[:20]}...")
        print(f"   To: {tx.get('to', 'N/A')[:20]}...")
        print(f"   Hash: {tx.get('hash', 'N/A')[:20]}...\n")

# Test with the latest block
find_high_value_transactions(latest_block, threshold_eth=1.0)

# YOUR TURN: Try different thresholds
# find_high_value_transactions(latest_block, threshold_eth=10.0)
# find_high_value_transactions(latest_block, threshold_eth=0.1)

### Challenge 2: Calculate Total Gas Spent

Calculate the total gas fees spent by all transactions in a block (in ETH).

In [ ]:
def calculate_block_gas_fees(block: Dict[str, Any]) -> None:
    """
    Calculate total gas fees for all transactions in a block.
    
    Hint: Total Fee = gas_used * effective_gas_price for each transaction
    """
    print("\n💰 BLOCK GAS FEE ANALYSIS\n")
    print("=" * 80)
    
    block_number = block.get('number', 'N/A')
    print(f"Block #{block_number:,}")
    
    gas_used = block.get('gasUsed', 0)
    base_fee_per_gas = block.get('baseFeePerGas', 0)
    
    print(f"\nTotal Gas Used: {gas_used:,}")
    print(f"Base Fee Per Gas: {base_fee_per_gas / 10**9:.2f} Gwei")
    
    # Approximate calculation using base fee
    # (In reality, each transaction may have different priority fees)
    base_fee_total_wei = gas_used * base_fee_per_gas
    base_fee_total_eth = base_fee_total_wei / 10**18
    
    print(f"\n🔥 Base Fee Burned: {base_fee_total_eth:.6f} ETH")
    print(f"   ≈ ${base_fee_total_eth * 2000:,.2f} USD")
    
    # Estimate priority fees (typically 5-10% of base fee)
    priority_fee_estimate = base_fee_total_eth * 0.07  # 7% estimate
    total_fees_eth = base_fee_total_eth + priority_fee_estimate
    
    print(f"\n💸 Priority Fees (estimated): {priority_fee_estimate:.6f} ETH")
    print(f"\n📊 Total Block Fees: {total_fees_eth:.6f} ETH")
    print(f"   ≈ ${total_fees_eth * 2000:,.2f} USD")
    
    # Per-transaction average
    tx_count = len(block.get('transactions', []))
    if tx_count > 0:
        avg_fee_eth = total_fees_eth / tx_count
        print(f"\n📈 Average Fee Per Transaction: {avg_fee_eth:.6f} ETH")
        print(f"   ≈ ${avg_fee_eth * 2000:.2f} USD")
    
    print("\n" + "=" * 80)
    
    # Educational note
    print("\n💡 Note: Base fees are burned (removed from circulation),")
    print("   while priority fees are paid to validators as rewards.")

# Test with latest block
calculate_block_gas_fees(latest_block)

# YOUR TURN: Fetch a different block and analyze it
# specific_block = get_block_by_number(18000000)
# calculate_block_gas_fees(specific_block)

### Challenge 3: Compare Transaction Costs

Create a function that estimates transaction costs for different gas price scenarios.

In [ ]:
def compare_transaction_costs(gas_units: int = 21000) -> None:
    """
    Compare transaction costs across different gas price scenarios.
    
    Args:
        gas_units: Number of gas units (21000 for simple transfer)
    """
    print(f"\n💸 TRANSACTION COST COMPARISON")
    print(f"Operation using {gas_units:,} gas units\n")
    print("=" * 80)
    
    scenarios = [
        ("Very Low (Off-peak)", 5, "⏰ 3-5 minutes"),
        ("Low", 10, "⏱️  1-2 minutes"),
        ("Normal", 25, "⚡ 30-60 seconds"),
        ("High", 50, "🚀 15-30 seconds"),
        ("Very High (Urgent)", 100, "💨 < 15 seconds"),
        ("Extreme (NFT drop)", 200, "🔥 Immediate"),
    ]
    
    print(f"{'Scenario':<25} {'Gas Price':>12} {'Cost (ETH)':>15} {'Cost (USD)':>15} {'Est. Time':>20}")
    print("-" * 95)
    
    for scenario, gas_price_gwei, time_est in scenarios:
        gas_price_wei = gas_price_gwei * 10**9
        cost_wei = gas_units * gas_price_wei
        cost_eth = cost_wei / 10**18
        cost_usd = cost_eth * 2000
        
        print(f"{scenario:<25} {gas_price_gwei:>10} Gwei {cost_eth:>15.6f} ${cost_usd:>14.2f} {time_est:>20}")
    
    print("\n" + "=" * 80)
    print("\n💡 Key Takeaways:")
    print("""
1. Gas price directly determines transaction speed and cost
2. 20x higher gas price = 20x higher cost (same operation)
3. During network congestion, costs can spike dramatically
4. Users must balance cost vs. urgency
5. This complexity is a major UX challenge for mainstream adoption
    """)
    
    print("\n🎯 Wallet Design Question:")
    print("How would YOU design an interface to help users make this decision?")
    print("Consider: Simplicity vs. Control, Cost awareness, Time urgency")

# Test with simple transfer
compare_transaction_costs(21000)

# YOUR TURN: Try with different operation costs
print("\n\n" + "="*80)
print("YOUR TURN: Compare costs for a Uniswap swap (150,000 gas)")
print("="*80)
# compare_transaction_costs(150000)

## Summary

In this notebook, you learned:

1. **Transaction Structure**: The key components of blockchain transactions and how they work
2. **Blockchain Data Access**: How to connect to Ethereum and query real-time data
3. **Transaction Analysis**: How to parse and interpret transaction details
4. **Gas Mechanics**: How gas prices work and affect transaction costs
5. **UX Challenges**: The significant gap between raw blockchain data and user-friendly interfaces

### Key Insights:

- Raw blockchain data is highly technical and intimidating to average users
- Wallets abstract away complexity through intelligent design
- Gas price variability creates decision paralysis for users
- One wrong character in an address means permanent loss of funds
- The biggest barrier to blockchain adoption is UX, not technology

### Next Steps:

- Explore different wallet implementations (MetaMask, Coinbase Wallet, Rainbow)
- Study ENS (Ethereum Name Service) for human-readable addresses
- Research account abstraction (EIP-4337) for improved UX
- Investigate Layer 2 solutions that reduce transaction costs
- Design your own wallet interface that balances simplicity with power

### Further Reading:

- [Ethereum Yellow Paper](https://ethereum.github.io/yellowpaper/paper.pdf) - Technical specification
- [EIP-1559](https://eips.ethereum.org/EIPS/eip-1559) - Fee market change
- [Web3.py Documentation](https://web3py.readthedocs.io/) - Python library
- [Etherscan](https://etherscan.io/) - Blockchain explorer
- [Gas Tracker](https://etherscan.io/gastracker) - Real-time gas prices